# Calculating the USA Today Diversity Index

1. What is the USAT Diversity Index?
1. Getting the data
1. processing the data with Excel
1. processing the data with Python


## What is the USAT Diversity Index

* Created by Phil Meyer (UNC) and Shawn McIntosh (USAT) in 1991
* [Reviewed in 2001](http://www.unc.edu/~pmeyer/carstat/tools.html#updating) after Census changed race question. Equation essentially unchanged.

### The equation

Diversity = 1-((_WhitePct_<sup>2</sup> + _BlackPct_<sup>2</sup> + _NativeAmericanPct_<sup>2</sup> + _AsianPct_<sup>2</sup> + _PacificIslanderPct_<sup>2</sup>) * (_HispanicPct_<sup>2</sup> + _NonHispanicPct_<sup>2</sup> ))



## Getting the data

The relevant data for these is in two tables
* [B02001 - Race](http://censusreporter.org/tables/B02001/)
* [B03003 - Hispanic or Latino Origin](http://censusreporter.org/tables/B03003/)

Census Reporter makes it easy to get Census Data for a variety of geographies. The links above take you to an overview page for each table, where you can enter the geography or geographies which you're analyzing. 

If you work on this stuff for a while, you'll become familiar with the key _geoids_ (geographic identifiers) for the places you study most. You can learn some more about geoids and other aspects of how the Census Bureau deals with geography on [Census Reporter's geography help page](http://censusreporter.org/topics/geography/).

Another thing, if you work on this stuff for a while, is that you might just start "hacking" the URLs instead of going through the Census Reporter GUI. For example, say you've gotten the page for **B02001** for all states in the US:

    http://censusreporter.org/data/table/?table=B02001&geo_ids=040|01000US&primary_geo_id=01000US

To go from that page to _B03003 - Hispanic or Latino Origin_, you just have to change the value for table:

    http://censusreporter.org/data/table/?table=B03003&geo_ids=040|01000US&primary_geo_id=01000US

You can download the data from those pages in many formats, including Excel. Note that the Excel files do not include verbal labels for columns, because some of the column names are extremely long, or are nested so that making the individual column names clear would be clumsy. The full information about column names is included in a `JSON` file which is part of the download, but for your reference, these are the relevant columns for computing the diversity index:

* B02001002 White alone
* B02001003 Black or African American alone
* B02001004 American Indian and Alaska Native alone
* B02001005 Asian alone
* B02001006 Native Hawaiian and Other Pacific Islander alone

* B03003002 Not Hispanic or Latino
* B03003003 Hispanic or Latino

Since you need to compute percentages, note that, as is standard, the first column in each table is the "total" column, or the denominator. The values should be the same, because the _universe_ for both tables is the same. If you find that they aren't, you've done something wrong.

* B02001001 Total population
* B03003001 Total population


## Processing the data with Excel

Visit the Census Reporter pages for the two tables and select your geographies of interest. Choose "Excel" from the "Download data" menu. Then merge the two spreadsheets into one.

If that sounds tedious, here's a trick: if you know the URL, you can include multiple table_ids in a special URL which downloads the Excel file directly. Here's the URL for these two tables for all states in the US:

    https://api.censusreporter.org/1.0/data/download/latest?format=xlsx&table_ids=B02001,B03003&geo_ids=040|01000US
    
If you don't use this URL, you'll need to get the columns all together in one spreadsheet. Leave the margin of error columns behind for now. Then add columns to compute the percentage of each race and hispanic/non-hispanic. Finally, add a column which implements the diversity index as shown above.


## Processing the data with python

Most popular data analysis tools for python can open Excel files. However, if you're comfortable programming in python, you may as well also take advantage of Census Reporter's API and some utility code we've got here for you. (Note that Census Reporter API will only return up to 3500 geographies in a single call.)

The code offers a generalized function to get a `pandas` dataframe for any combination of tables and geoids, and then a function which, given a `DataFrame` which is assumed to have the right columns, returns a `Series` which is the diversity index.

In [1]:
import pandas as pd
from urllib2 import urlopen
import json

def dataframe_from_api(tables=None,geoids=None,include_moe=False):
    """Given data as it comes back from the Census Reporter API, produce a dataframe with 
       columns for each Census column, plus 'name'. the Index will be the geoid.
       
       Table IDs and geoids can be passed as either lists or comma-separated strings.
    """
    if tables is None: tables = ['B01001'] # default to sex by age
    if geoids is None: geoids = ['040|01000US'] # default to all states in the US
        
    try: 
        tables = ','.join(tables)
    except:
        pass
    try:
        geoids = ','.join(geoids)
    except:
        pass
    url = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids={}&geo_ids={}'.format(tables,geoids)
    resp = urlopen(url)
    data = json.load(resp)
    d = { }
    for geoid in data['data']:
        d[geoid] = { 'name': data['geography'][geoid]['name'] }
        for table_id, table_data in data['data'][geoid].iteritems():
            for column, value in table_data['estimate'].iteritems():
                d[geoid][column] = value
                if include_moe:
                    d[geoid]["{}_moe".format(column)] = table_data['error'][column]
    return pd.DataFrame.from_dict(d,orient='index')

def compute_diversity(df):
    white_pct = df['B02001002'] / df['B02001001']
    black_pct = df['B02001003'] / df['B02001001']
    amerind_pct = df['B02001004'] / df['B02001001']
    asian_pct = df['B02001005'] / df['B02001001']
    nhpi_pct = df['B02001006'] / df['B02001001']
    nonhisp_pct = df['B03003002'] / df['B03003001']
    hisp_pct = df['B03003003'] / df['B03003001']
    return 1-( 
              (white_pct**2 + black_pct**2 + amerind_pct**2 + asian_pct**2 + nhpi_pct**2 ) * 
              (hisp_pct**2 + nonhisp_pct**2)
           )


In [2]:
df = dataframe_from_api(['B02001', 'B03003'],['040|01000US'])
df['diversity'] = compute_diversity(df)

In [3]:
df[['name','diversity']].sort_values(by='diversity',ascending=False)

,name,diversity
04000US15,Hawaii,0.821999
04000US06,California,0.789932
04000US35,New Mexico,0.727072
04000US32,Nevada,0.715009
04000US48,Texas,0.694184
04000US36,New York,0.692731
04000US11,District of Columbia,0.673453
04000US34,New Jersey,0.665889
04000US24,Maryland,0.649323
04000US04,Arizona,0.644045


In [4]:
def compute_diversity2(df):
    """An alternative computation, out of curiosity, to see if the original overweights Hispanic diversity
       Depends on table B03002 - Hispanic or Latino Origin by Race
       B03002003 - NH White
       B03002004 - NH Black
       B03002005 - NH Native American
       B03002006 - NH Asian
       B03002007 - NH Pacific Islander
       B03002012 - Hispanic (all races)
    """
    white_pct = df['B03002003'] / df['B03002001']
    black_pct = df['B03002004'] / df['B03002001']
    amerind_pct = df['B03002005'] / df['B03002001']
    asian_pct = df['B03002006'] / df['B03002001']
    nhpi_pct = df['B03002007'] / df['B03002001']
    hisp_pct = df['B03002012'] / df['B03002001']
    return 1-( 
              (white_pct**2 + black_pct**2 + amerind_pct**2 + asian_pct**2 + nhpi_pct**2 + hisp_pct**2)
           )
    
    
df2 = dataframe_from_api(['B03002'],['040|01000US'])
df['diversity2'] = compute_diversity2(df2)
df[['name','diversity','diversity2']].sort_values(by='diversity',ascending=False)

,name,diversity,diversity2
04000US15,Hawaii,0.821999,0.794839
04000US06,California,0.789932,0.682362
04000US35,New Mexico,0.727072,0.614378
04000US32,Nevada,0.715009,0.646401
04000US48,Texas,0.694184,0.646790
04000US36,New York,0.692731,0.620758
04000US11,District of Columbia,0.673453,0.632381
04000US34,New Jersey,0.665889,0.617191
04000US24,Maryland,0.649323,0.627366
04000US04,Arizona,0.644045,0.587761


In [5]:
df.columns

Index([ u'B02001002',  u'B02001003',  u'B02001001',  u'B02001006',
             u'name',  u'B02001004',  u'B02001005',  u'B02001008',
        u'B02001009',  u'B02001010',  u'B03003001',  u'B03003003',
        u'B03003002',  u'B02001007',  u'diversity', u'diversity2'],
      dtype='object')